In [2]:
import duckdb
import pandas

In [4]:
# Read directly from GitHub and assign to an object using using Python
attribution_studies = pandas.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-08-12/attribution_studies.csv')
attribution_studies_raw = pandas.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-08-12/attribution_studies_raw.csv')
attribution_studies

,event_name,event_period,event_year,study_focus,iso_country_code,cb_region,event_type,classification,summary_statement,publication_year,citation,source,rapid_study,link
0,European summer heatwave,2003,2003,Event,NaN,Europe,Heat,More severe or more likely to occur,"""We estimate it is very likely (confidence lev...",2004.0,"Stott, P. et al., 2004: Human contribution to ...",Nature,No,https://www.nature.com/nature/journal/v432/n70...
1,Global temperature extremes,since 1950,NaN,Trend,NaN,Global,Heat,More severe or more likely to occur,"""Comparing these observations with climate mod...",2005.0,"Christidis, N. et al., 2005: Detection of chan...",Geophysical Research Letters,No,https://agupubs.onlinelibrary.wiley.com/doi/fu...
2,Record warm autumn in Europe,2006,2006,Event,NaN,Europe,Heat,More severe or more likely to occur,"""Global warming has made a warm autumn like th...",2007.0,"Van Oldenborgh, G-J. et al., 2007: How unusual...",Climate of the Past,No,http://www.clim-past.net/3/659/2007/cp-3-659-2...
3,Increasing frequency of 'very warm' Northern h...,1860-2009,NaN,Trend,NaN,Northern hemisphere,Heat,More severe or more likely to occur,"""We detect the dominant influence of anthropog...",2007.0,"Jones, G. et al., 2007: Human contribution to ...",Journal of Geophysical Research: Atmospheres,No,https://agupubs.onlinelibrary.wiley.com/doi/fu...
4,Moscow summer heatwave,2010,2010,Event,RUS,Europe,Heat,More severe or more likely to occur,"""For July temperature in Moscow, we estimate.....",2011.0,"Rahmstorf, S. & Coumou, D. 2011: Increase of e...",Proceedings of the National Academy of Sciences,No,http://www.pnas.org/content/108/44/17905.abstract
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,China growing season hot and dry events,mid 1990s,Mid-1990s,Event,CHN,Eastern and south-eastern Asia,Compound,More severe or more likely to occur,"""Anthropogenic forcing changes can explain 60%...",2024.0,"Su Q. et al., 2024: Anthropogenic Influence on...",Advances in Atmospheric Sciences,No,https://doi.org/10.1007/s00376-023-2319-z
740,China hot-wet events,1979-2014,NaN,Trend,CHN,Eastern and south-eastern Asia,Compound,"Decrease, less severe or less likely to occur","""Over the past 40 years, anthropogenic activit...",2024.0,"Yao H. et al., 2024: Changes caused by human a...",Communications Earth & Environment,No,https://doi.org/10.1038/s43247-024-01625-y
741,Heat extremes behind bleaching of the Great Ba...,"2020, 2024","2017, 2020, 2024",Event,AUS,Australia and New Zealand,Impact,More severe or more likely to occur,"""Climate model analysis confirms that human in...",2024.0,"Henley B.J. et al., 2024: Highest ocean heat i...",Nature,No,https://doi.org/10.1038/s41586-024-07672-x
742,Heat-related neonatal deaths in low- and middl...,2001-2019,NaN,Trend,NaN,Global,Impact,More severe or more likely to occur,"""Climate change was responsible for 32% (range...",2024.0,"Dimitrova. A. et al., 2024: Temperature-relate...",Nature Communications,No,https://www.nature.com/articles/s41467-024-498...


In [5]:
# How do attribution study publications evolve over time? What about rapid attribution studies?

In [7]:
cn = duckdb.connect("attribution_studies_data.db")
cn.execute("""
            INSTALL https;
            LOAD https;
            INSTALL excel;
            LOAD excel;
            """)

sql = """
        create or replace table attribution_data
        as 
        select event_name, 
                event_period, 
                event_year, 
                study_focus, 
                iso_country_code, 
                cb_region, 
                event_type, 
                classification, 
                summary_statement, 
                publication_year, 
                citation, 
                source, 
                rapid_study, 
                link    
        from attribution_studies
"""
cn.sql(sql)

In [25]:
# What type of extreme event is more frequently the subject of an attribution study?
cn.sql("select count(*) as event_count, event_type from attribution_data group by event_type order by 1 desc")

┌─────────────┬──────────────────┐
│ event_count │    event_type    │
│    int64    │     varchar      │
├─────────────┼──────────────────┤
│         207 │ Heat             │
│         177 │ Rain & flooding  │
│         106 │ Drought          │
│          61 │ Storm            │
│          57 │ Cold, snow & ice │
│          33 │ Impact           │
│          33 │ Compound         │
│          31 │ Wildfire         │
│          25 │ Oceans           │
│           7 │ Atmosphere       │
│           4 │ Sunshine         │
│           3 │ River flow       │
├─────────────┴──────────────────┤
│ 12 rows              2 columns │
└────────────────────────────────┘

In [27]:
# In which regions are most studies focused?
cn.sql("select count(*) as region_count, cb_region from attribution_data group by cb_region order by 1 desc")

┌──────────────┬──────────────────────────────────┐
│ region_count │            cb_region             │
│    int64     │             varchar              │
├──────────────┼──────────────────────────────────┤
│          165 │ Europe                           │
│          161 │ Eastern and south-eastern Asia   │
│          137 │ Northern America                 │
│           63 │ Global                           │
│           56 │ Sub-Saharan Africa               │
│           49 │ Australia and New Zealand        │
│           36 │ Central and southern Asia        │
│           32 │ Latin America and the Caribbean  │
│           15 │ Northern hemisphere              │
│           11 │ Arctic                           │
│            8 │ Oceania                          │
│            8 │ Northern Africa and western Asia │
│            3 │ Antarctica                       │
├──────────────┴──────────────────────────────────┤
│ 13 rows                               2 columns │
└───────────

In [ ]:
# Is there a trend in how climate change influences different types of extreme weather?